In [2]:
import pandas as pd
import numpy as np

path =('../../dataset/')

# 장르 결측치 처리 된 거 이후부터 시작!
movies_data = pd.read_csv('movies_genre_crawling.csv')
movies_data.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [3]:
movies_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9742 non-null   int64 
 1   title    9742 non-null   object
 2   genres   9742 non-null   object
dtypes: int64(1), object(2)
memory usage: 228.5+ KB


In [4]:
movie_dataset = movies_data.copy()

In [16]:
links_data = pd.read_csv(path+'links.csv')
links_data.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [17]:
movies = pd.merge(movie_dataset, links_data, on='movieId')

## 크롤링 준비
imdb_id를 url에 집어 넣는 형태로 변환

In [5]:
def create_imdb_url(imdb_id):
    # imdbId를 문자열로 변환하고, 7자리가 되도록 앞에 0을 추가
    imdb_id_str = str(imdb_id).zfill(7)
    # URL 생성
    url = f"https://www.imdb.com/title/tt{imdb_id_str}/"
    return url

In [6]:
# 예시
imdb_id = 114709
url = create_imdb_url(imdb_id)
print(url)

https://www.imdb.com/title/tt0114709/


## 1개로 연습

In [7]:
from selenium import webdriver

browser = webdriver.Chrome()
browser.get('https://www.google.com/')

In [8]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(browser.page_source, 'html.parser')

In [14]:
names = movie_dataset['title'].tolist()
name = names[0]

In [18]:
imdb_id = movies.loc[movies['title'] == name, 'imdbId'].values[0]
imdb_id

114709

In [19]:
imdb_url = create_imdb_url(imdb_id)

In [20]:
# IMDb 페이지로 이동
browser.get(imdb_url)

In [24]:
soup = BeautifulSoup(browser.page_source, 'html.parser')

In [27]:
# 스크롤 내리기
from selenium.webdriver.common.keys import Keys

body = browser.find_elements('css selector', 'body')[0]
body.send_keys(Keys.PAGE_DOWN)
body.send_keys(Keys.PAGE_DOWN)
body.send_keys(Keys.PAGE_DOWN)
body.send_keys(Keys.PAGE_DOWN)
body.send_keys(Keys.PAGE_DOWN)

soup = BeautifulSoup(browser.page_source, 'html.parser')

import time
time.sleep(1)

In [32]:
soup = BeautifulSoup(browser.page_source, 'html.parser')

text_temp = soup.select('#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > div > section > div > div.sc-9178d6fe-1.fqVKQb.ipc-page-grid__item.ipc-page-grid__item--span-2 > section > div.sc-ab73d3b3-0.iMuSjK > div.ipc-overflowText.ipc-overflowText--pageSection.ipc-overflowText--base > div > div > div')
text_temp[0].text

'A little boy named Andy loves to be in his room, playing with his toys, especially his doll named "Woody". But, what do the toys do when Andy is not with them, they come to life. Woody believes that his life (as a toy) is good. However, he must worry about Andy\'s family moving, and what Woody does not know is about Andy\'s birthday party. Woody does not realize that Andy\'s mother gave him an action figure known as Buzz Lightyear, who does not believe that he is a toy, and quickly becomes Andy\'s new favorite toy. Woody, who is now consumed with jealousy, tries to get rid of Buzz. Then, both Woody and Buzz are now lost. They must find a way to get back to Andy before he moves without them, but they will have to pass through a ruthless toy killer, Sid Phillips. —John Wiggins'

# 전체 데이터셋을 대상으로 크롤링 후 저장!

In [33]:
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
from bs4 import BeautifulSoup
import time

browser = webdriver.Chrome()
browser.get('https://www.google.com/')

soup = BeautifulSoup(browser.page_source, 'html.parser')

names = movie_dataset['title'].tolist()

In [34]:
from tqdm import tqdm

#진행상황 확인을 위한
total_movies = len(names)

un_storyline = []

for name in tqdm(names, desc="Crawling movies", unit="movie"):
    soup = BeautifulSoup(browser.page_source, 'html.parser')
    
    imdb_id = movies.loc[movies['title'] == name, 'imdbId'].values[0]
    imdb_url = create_imdb_url(imdb_id)
    # IMDb 페이지로 이동
    browser.get(imdb_url)
    
    soup = BeautifulSoup(browser.page_source, 'html.parser')
   
    # 스크롤 내리기
    body = browser.find_elements('css selector', 'body')[0]
    body.send_keys(Keys.PAGE_DOWN)
    body.send_keys(Keys.PAGE_DOWN)
    body.send_keys(Keys.PAGE_DOWN)
    body.send_keys(Keys.PAGE_DOWN)
    body.send_keys(Keys.PAGE_DOWN)

    soup = BeautifulSoup(browser.page_source, 'html.parser')
    
    time.sleep(1)
    soup = BeautifulSoup(browser.page_source, 'html.parser')

    try:
        text_temp = soup.select('#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > div > section > div > div.sc-9178d6fe-1.fqVKQb.ipc-page-grid__item.ipc-page-grid__item--span-2 > section > div.sc-ab73d3b3-0.iMuSjK > div.ipc-overflowText.ipc-overflowText--pageSection.ipc-overflowText--base > div > div > div')
        storyline = text_temp[0].text

        movie_dataset.loc[movie_dataset['title'] == name, 'storyline'] = storyline

    except IndexError:
        un_storyline.append(name)
        
        

Crawling movies:  14%|███████▏                                             | 1321/9742 [3:24:08<21:41:20,  9.27s/movie]


WebDriverException: Message: disconnected: not connected to DevTools
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: chrome=119.0.6045.160)
Stacktrace:
	GetHandleVerifier [0x00007FF68D6282B2+55298]
	(No symbol) [0x00007FF68D595E02]
	(No symbol) [0x00007FF68D4505AB]
	(No symbol) [0x00007FF68D43E19B]
	(No symbol) [0x00007FF68D43DDF0]
	(No symbol) [0x00007FF68D4524A1]
	(No symbol) [0x00007FF68D4CA18F]
	(No symbol) [0x00007FF68D4B1E83]
	(No symbol) [0x00007FF68D48670A]
	(No symbol) [0x00007FF68D487964]
	GetHandleVerifier [0x00007FF68D9A0AAB+3694587]
	GetHandleVerifier [0x00007FF68D9F728E+4048862]
	GetHandleVerifier [0x00007FF68D9EF173+4015811]
	GetHandleVerifier [0x00007FF68D6C47D6+695590]
	(No symbol) [0x00007FF68D5A0CE8]
	(No symbol) [0x00007FF68D59CF34]
	(No symbol) [0x00007FF68D59D062]
	(No symbol) [0x00007FF68D58D3A3]
	BaseThreadInitThunk [0x00007FFA8CD4257D+29]
	RtlUserThreadStart [0x00007FFA8E66AA58+40]


In [36]:
movie_dataset.loc[1320:1325]

,movieId,title,genres,storyline
1320,1779,Sphere (1998),Sci-Fi|Thriller,"1000 feet below the ocean, navy divers discove..."
1321,1783,Palmetto (1998),Crime|Drama|Mystery|Romance|Thriller,NaN
1322,1784,As Good as It Gets (1997),Comedy|Drama|Romance,NaN
1323,1785,King of New York (1990),Crime|Thriller,NaN
1324,1791,Twilight (1998),Crime|Drama|Thriller,NaN
1325,1792,U.S. Marshals (1998),Action|Crime|Thriller,NaN


In [37]:
movie_dataset.to_csv('movies_genre_storyline_ing.csv', index=False)
